[Home](../../README.md)

### Feature Engineering

This Jupyter Notepad is a selection of data engineering processes you can apply to your data before model training to maximise the performance of your machine learning model. For this demonstration we will engineer new or improved features for the diabetes data you previously wrangled.

#### Feature Engineering Process
- Deriving new variables from existing ones
    - Encoding categorical features
    - Calculating new features from existing features
- Combining features/feature interactions
- Identifying the most relevant features for the model
- Transforming Features
  - [Dividing Data into categories](https://web.ma.utexas.edu/users/mks/statmistakes/dividingcontinuousintocategories.html)
  - Mathematical transformations (for example logarithmic transformations). Logarithmic transformations are a powerful tool in the world of statistical analysis. They are often used to transform data that exhibit skewness or other irregularities, making it easier to analyze, visualize, and interpret the results.
- Creating Domain-Specific Features that incorporating knowledge from the specific domain to create features that capture important characteristics of the data.

#### Load the required dependencies

In [44]:
# Import frameworks
import pandas as pd

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [45]:
data_frame = pd.read_csv("2.2.1.wrangled_data.csv")

####  Deriving new variables from existing ones

##### Encoding categorical variables

Data Encoding converts textual data into numerical format, so that it can be used as input for algorithms to process. The reason for encoding is that most machine learning algorithms work with numbers and not with text or categorical variables.

To encode the 'SEX' column you will assigning a number value to the gender. Because the data set only provides 2 values we will use -1 and 1.

In [46]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: -1 if gender.lower() == 'male' else 1 if gender.lower() == 'female' else None)
print(data_frame['SEX'].head())

0    1
1    1
2   -1
3   -1
4   -1
Name: SEX, dtype: int64


##### Calculating Age

In the context of medical diagnosis of a lifestyle disease a persons date of birth has limited influence on the target. However, their age is highly relevant. So we will convert two dates into a age. You could consider further encoding this into age brackets.

In [47]:
# Convert the 'DoB' and 'DoTest' columns to datetime
data_frame['DoB'] = pd.to_datetime(data_frame['DoB'], format='%d/%m/%Y')
data_frame['DoT'] = pd.to_datetime(data_frame['DoT'], format='%d/%m/%Y')

# Calculate the year difference
data_frame['Age'] = ((data_frame['DoT'] - data_frame['DoB']).dt.days  / 365.25).round()

# Print the result
print(data_frame[['DoB', 'DoT', 'Age']])

           DoB        DoT   Age
0   2001-06-17 2024-10-09  23.0
1   1995-06-10 2024-08-16  29.0
2   1981-03-11 2024-03-08  43.0
3   2002-05-15 2024-05-06  22.0
4   2001-06-21 2024-07-20  23.0
..         ...        ...   ...
422 1955-04-06 2024-04-12  69.0
423 1962-06-22 2024-09-14  62.0
424 1966-08-26 2024-11-27  58.0
425 1980-04-28 2024-04-03  44.0
426 1987-06-13 2024-06-09  37.0

[427 rows x 3 columns]


#### Combining features/feature interactions

While individual features can be powerful predictors, their interactions often carry even more information. Feature interaction engineering is the process of creating new features that represent the interaction between two or more features.

In this, case some domain knowledge and data analysis have informed you that the BMI and AGE are risk multipliers (the greater the age and the greater the BMI the greater the feature). From this we can  risk value based on the feature interactions.

In [48]:
# Calculate the year difference and round to an integer
data_frame['Age'] = ((data_frame['DoT'] - data_frame['DoB']).dt.days / 365.25).round().astype(int)

# Create the 'Risk' column
data_frame['Risk'] = data_frame['BMI'] * data_frame['Age']

# Calculate the percentage of the maximum risk
data_frame['Risk%'] = (data_frame['Risk'] / data_frame['Risk'].max()).round(2)

# Print the result
print(data_frame[['Age', 'BMI', 'Risk%']])

     Age       BMI  Risk%
0     23  0.115385   0.05
1     29  0.119231   0.06
2     43  0.134615   0.10
3     22  0.138462   0.05
4     23  0.146154   0.06
..   ...       ...    ...
422   69  0.846154   1.00
423   62  0.876923   0.93
424   58  0.884615   0.88
425   44  0.892308   0.67
426   37  0.896154   0.57

[427 rows x 3 columns]


#### Transforming Features

Filtering is like applying the where clause in a database. It is widely used and can help when you need to work on a specific subset of your data. For our use case, let us filter the data to only include rows where the 'SEX' is 'Male'. There is no method call for this, we can just use conditional indexing to fulfil our purpose.

In this, case some domain knowledge and data analysis have informed you that there is 'bimodality' in the data and males and females have a different trends. 

In [49]:
# Filter the data to -1 only
data_frame = data_frame[data_frame['SEX'] == -1]

# Print the result
print(data_frame[['Age', 'SEX', 'Target']])

     Age  SEX    Target
2     43   -1  0.215385
3     22   -1  0.249231
4     23   -1  0.200000
5     26   -1  0.095385
8     19   -1  0.360000
..   ...  ...       ...
416   43   -1  0.784615
419   57   -1  0.769231
420   55   -1  0.732308
422   69   -1  0.667692
426   37   -1  0.735385

[228 rows x 3 columns]


#### Creating Domain-Specific Features

Domain knowledge is about understanding the domain or subject area of the data. In This case the domain is 'health' and more specifically   'Epidemiology' which is the study of how often diseases occur in different groups of people and why.

The column called '1st Degree Relatives' is a domain specific feature as is records the number of family members in the individuals direct bloodline who have developed type 2 adult onset diabetes. Domain specific knowledge, is that Family history of disease in first degree relatives is a major risk factor, especially for premature events.

First we will convert we will convert the FDR value to a risk percentage, because the risk can never be 0 (will never happen) or 100% (will definitely happen) we will scale the result between 0.15 and 0.95.

In [50]:
# Calculate the family history risk
data_frame['FHRisk'] = (data_frame['FDR'] / data_frame['FDR'].max())

# Scale the result between 0.15 and 0.85
min_val = 0.15
max_val = 0.85
data_frame['FHRisk'] = (((data_frame['FHRisk'] - data_frame['FHRisk'].min()) / (data_frame['FHRisk'].max() - data_frame['FHRisk'].min())) * (max_val - min_val) + min_val).round(2)

# Print the result
print(data_frame[['Age', 'FDR', 'FHRisk']])

     Age   FDR  FHRisk
2     43  0.50    0.62
3     22  0.50    0.62
4     23  0.50    0.62
5     26  0.50    0.62
8     19  0.00    0.15
..   ...   ...     ...
416   43  0.00    0.15
419   57  0.25    0.38
420   55  0.50    0.62
422   69  0.50    0.62
426   37  0.00    0.15

[228 rows x 3 columns]


Then to make it even more meaningful, we will combine it with the `Risk` feature we engineered using the `AGE` and `BMI` features to create a combined risk 'interaction feature' that captures real-world relationships between the features.

Again we will scale the result between 0.15 and 0.95.

In [51]:
data_frame['CombRisk'] = (data_frame['FHRisk'] * data_frame['Risk%']).round(2)

min_val = 0.15
max_val = 0.85
data_frame['CombRisk'] = (((data_frame['CombRisk'] - data_frame['CombRisk'].min()) / (data_frame['CombRisk'].max() - data_frame['CombRisk'].min())) * (max_val - min_val) + min_val).round(2)

# Print the result
print(data_frame[['Age', 'Risk%', 'FHRisk', 'CombRisk']])

     Age  Risk%  FHRisk  CombRisk
2     43   0.10    0.62      0.21
3     22   0.05    0.62      0.17
4     23   0.06    0.62      0.18
5     26   0.07    0.62      0.18
8     19   0.05    0.15      0.15
..   ...    ...     ...       ...
416   43   0.58    0.15      0.24
419   57   0.79    0.38      0.48
420   55   0.78    0.62      0.69
422   69   1.00    0.62      0.85
426   37   0.57    0.15      0.24

[228 rows x 4 columns]


In [52]:
# Create interaction features
data_frame['BMI_Age'] = data_frame['BMI'] * data_frame['Age']
data_frame['BP_Age'] = data_frame['BP'] * data_frame['Age']
data_frame['BMI_BP'] = data_frame['BMI'] * data_frame['BP']

# Print the result
print(data_frame[['BMI', 'BP', 'BMI_BP']])



          BMI        BP    BMI_BP
2    0.134615  0.363636  0.048951
3    0.138462  0.493506  0.068332
4    0.146154  0.246753  0.036064
5    0.146154  0.311688  0.045554
8    0.161538  0.363636  0.058741
..        ...       ...       ...
416  0.784615  0.441558  0.346454
419  0.811538  0.753247  0.611289
420  0.830769  0.701299  0.582617
422  0.846154  0.571429  0.483516
426  0.896154  0.701299  0.628472

[228 rows x 3 columns]


#### Save the wrangled and engineered data to CSV

In [55]:
data_frame.to_csv('../2.3.Model_Training/2.3.1.model_ready_data2.csv', index=False)